# Analyse du Data Drift — Credit Scoring

Ce notebook realise l'analyse de la **derive des donnees (data drift)** pour notre modele de scoring credit.

**Objectifs :**

- Comprendre ce qu'est le data drift et pourquoi c'est critique en production
- Analyser nos donnees avec **Evidently AI** (outil de reference pour le monitoring ML)
- Simuler differents scenarios de drift et mesurer leur impact
- Identifier les points de vigilance pour le monitoring en production


---

## 1. Qu'est-ce que le Data Drift ?

### Le probleme

Un modele ML est entraine sur des donnees historiques. En production, il recoit de **nouvelles donnees** pour faire ses predictions. Si ces nouvelles donnees ont des **distributions differentes** de celles d'entrainement, le modele risque de **perdre en fiabilite** — c'est le **data drift**.

### Analogie concrete

Imagine un modele de scoring entraine sur des clients francais avec des revenus moyens de 30 000 euros. Si en production il recoit soudainement des clients avec des revenus moyens de 80 000 euros (par exemple suite a l'ouverture du service a une clientele premium), la distribution du revenu a **derive**. Le modele n'a jamais vu ces profils — ses predictions deviennent moins fiables.

### Les types de drift

| Type              | Description                                          | Exemple                                            |
| ----------------- | ---------------------------------------------------- | -------------------------------------------------- |
| **Drift graduel** | Les distributions changent lentement au fil du temps | Inflation qui augmente les montants de credit      |
| **Drift soudain** | Changement brutal dans les donnees                   | Nouveau segment de clientele, crise economique     |
| **Feature shift** | Certaines variables specifiques changent             | Un fournisseur de score externe modifie son calcul |

### Comment detecter le drift ?

On compare la **distribution de reference** (donnees d'entrainement) avec la **distribution de production** (nouvelles donnees). Les tests statistiques les plus utilises :

- **Test de Kolmogorov-Smirnov (KS)** : mesure la distance maximale entre deux distributions. Si la p-value < 0.05, on considere qu'il y a drift.
- **Population Stability Index (PSI)** : mesure la stabilite d'une variable dans le temps.
- **Wasserstein distance** : mesure le "cout" pour transformer une distribution en une autre.

> **Important** : drift ne signifie pas forcement degradation du modele. Le modele peut etre robuste malgre un drift modere. Mais un drift significatif est un **signal d'alerte** qui necessite investigation.


---

## 2. Chargement des donnees


In [ ]:
import json
import sys
import warnings
from pathlib import Path

import joblib
import numpy as np
import pandas as pd

# Ajouter la racine du projet au path
sys.path.insert(0, str(Path.cwd().parent))

from evidently import Report
from evidently.presets import DataDriftPreset
from evidently.metrics import ValueDrift

from monitoring.drift import compute_drift_report, simulate_drift

warnings.filterwarnings("ignore", category=FutureWarning)

ARTIFACTS_DIR = Path("../artifacts")
DATA_DIR = Path("../data")

print("Imports OK")

### Donnees de reference (entrainement) vs production (test)

- **Reference** : `train_preprocessed.csv` — les donnees sur lesquelles le modele a ete entraine. C'est notre **baseline** : la distribution "normale" que le modele connait.
- **Production simulee** : `test_preprocessed.csv` — les donnees de test, qui simulent ce que le modele recevrait en production. Elles proviennent du meme dataset mais n'ont pas ete vues a l'entrainement.


In [ ]:
# Charger les donnees
reference = pd.read_csv(DATA_DIR / "train_preprocessed.csv")
production = pd.read_csv(DATA_DIR / "test_preprocessed.csv")

# Retirer les colonnes non-features
if "SK_ID_CURR" in reference.columns:
    reference = reference.drop("SK_ID_CURR", axis=1)
if "SK_ID_CURR" in production.columns:
    production = production.drop("SK_ID_CURR", axis=1)
if "TARGET" in reference.columns:
    reference = reference.drop("TARGET", axis=1)

# Garder uniquement les colonnes communes
common_cols = sorted(set(reference.columns) & set(production.columns))
reference = reference[common_cols]
production = production[common_cols]

print(f"Reference  : {reference.shape[0]:,} lignes, {reference.shape[1]} features")
print(f"Production : {production.shape[0]:,} lignes, {production.shape[1]} features")
print(f"\nApercu des 5 premieres features :")
reference.head(3)

In [ ]:
# Statistiques descriptives pour quelques features cles
key_features = [
    "EXT_SOURCE_1",
    "EXT_SOURCE_3",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "DAYS_BIRTH",
    "AMT_GOODS_PRICE",
]
key_features = [f for f in key_features if f in common_cols]

print("=== Stats Reference (train) ===")
display(reference[key_features].describe().round(3))

print("\n=== Stats Production (test) ===")
display(production[key_features].describe().round(3))

---

## 3. Detection de drift avec Evidently AI

**Evidently AI** est la bibliotheque de reference pour le monitoring ML. Elle genere des rapports interactifs qui comparent automatiquement les distributions de chaque feature entre la reference et la production.

Le `DataDriftPreset` applique le **test de Kolmogorov-Smirnov** (pour les variables numeriques) sur chaque colonne et determine si un drift est detecte (p-value < 0.05).


In [ ]:
# Echantillonner pour la rapidite (toutes les features, 5000 lignes max)
N_SAMPLES = 5000
ref_sample = reference.sample(n=min(N_SAMPLES, len(reference)), random_state=42)
prod_sample = production.sample(n=min(N_SAMPLES, len(production)), random_state=42)

print(f"Echantillon reference  : {len(ref_sample)} lignes")
print(f"Echantillon production : {len(prod_sample)} lignes")
print(f"Features analysees     : {len(common_cols)}")

In [ ]:
# Rapport Evidently : DataDriftPreset sur reference vs production
drift_report = Report([DataDriftPreset()])
snapshot = drift_report.run(ref_sample, prod_sample)

# Afficher le rapport interactif
snapshot

### Interpretation du rapport Evidently

Le rapport ci-dessus montre :

- **En haut** : le nombre total de features en drift vs stables
- **Pour chaque feature** : la p-value du test KS, la decision (drift ou non), et les distributions comparees

Si le rapport indique peu ou pas de drift entre train et test, c'est **normal** : les deux jeux proviennent du meme dataset Home Credit. Cela constitue notre **baseline saine**.


In [ ]:
# Extraire les resultats sous forme structuree
results_dict = snapshot.dict()

# Parser les metriques de drift par feature
drift_metrics = []
for m in results_dict["metrics"]:
    if m["config"]["type"] == "evidently:metric_v2:ValueDrift":
        drift_metrics.append(
            {
                "feature": m["config"]["column"],
                "method": m["config"]["method"],
                "p_value": m["value"],
                "threshold": m["config"]["threshold"],
                "drift_detected": m["value"] < m["config"]["threshold"],
            }
        )

df_drift = pd.DataFrame(drift_metrics).sort_values("p_value")

n_drifted = df_drift["drift_detected"].sum()
n_total = len(df_drift)
print(
    f"\nResultats Evidently : {n_drifted}/{n_total} features en drift ({n_drifted / n_total * 100:.1f}%)"
)
print(f"\nTop 10 features avec le plus de drift (p-value la plus basse) :")
df_drift.head(10)

In [ ]:
# Sauvegarder le rapport HTML pour partage
report_path = Path("../monitoring/drift_report_baseline.html")
snapshot.save_html(str(report_path))
print(f"Rapport HTML sauvegarde : {report_path}")

### Comparaison avec notre approche manuelle (scipy KS)

Notre module `monitoring/drift.py` utilise directement `scipy.stats.ks_2samp`. Verifions que les resultats sont coherents avec Evidently.


In [ ]:
# Notre approche manuelle
manual_report = compute_drift_report(ref_sample, prod_sample, top_n=len(common_cols))

# Comparer les deux approches
comparison = manual_report.merge(
    df_drift[["feature", "p_value"]].rename(columns={"p_value": "evidently_p_value"}),
    on="feature",
    how="inner",
)
comparison["manual_p_value"] = comparison["p_value"]
comparison["p_value_diff"] = abs(
    comparison["manual_p_value"] - comparison["evidently_p_value"]
)

print("Comparaison des p-values (scipy vs Evidently) :")
print(
    f"Correlation : {comparison['manual_p_value'].corr(comparison['evidently_p_value']):.4f}"
)
print(f"Difference moyenne : {comparison['p_value_diff'].mean():.6f}")
print("\n=> Les deux approches donnent des resultats quasi identiques.")
print("   Evidently utilise le meme test KS en interne, avec un rapport plus riche.")

---

## 4. Simulation de scenarios de drift

Pour comprendre comment le drift se manifeste, nous allons **simuler** 3 scenarios sur nos donnees de production et observer comment Evidently les detecte.

Les simulations sont realisees par notre module `monitoring/drift.py` :

- **Drift graduel** : ajout de bruit gaussien sur toutes les features
- **Drift soudain** : decalage brutal des 20 premieres features
- **Feature shift** : multiplication de 10% des features aleatoires


In [ ]:
scenarios = [
    {"name": "Aucun drift", "drift_type": "none", "intensity": 0.0},
    {"name": "Drift graduel (faible)", "drift_type": "gradual", "intensity": 0.1},
    {"name": "Drift graduel (fort)", "drift_type": "gradual", "intensity": 0.5},
    {"name": "Drift soudain", "drift_type": "sudden", "intensity": 0.3},
    {"name": "Feature shift", "drift_type": "feature_shift", "intensity": 0.3},
]

scenario_results = []

for sc in scenarios:
    # Simuler le drift sur les donnees de production
    drifted = simulate_drift(
        prod_sample, drift_type=sc["drift_type"], intensity=sc["intensity"]
    )

    # Lancer Evidently
    report = Report([DataDriftPreset()])
    snap = report.run(ref_sample, drifted)
    results = snap.dict()

    # Compter les features en drift
    n_drifted = sum(
        1
        for m in results["metrics"]
        if m["config"]["type"] == "evidently:metric_v2:ValueDrift"
        and m["value"] < m["config"]["threshold"]
    )
    n_features = sum(
        1
        for m in results["metrics"]
        if m["config"]["type"] == "evidently:metric_v2:ValueDrift"
    )

    scenario_results.append(
        {
            "Scenario": sc["name"],
            "Type": sc["drift_type"],
            "Intensite": sc["intensity"],
            "Features en drift": n_drifted,
            "Total features": n_features,
            "% drift": f"{n_drifted / n_features * 100:.0f}%"
            if n_features > 0
            else "N/A",
        }
    )

df_scenarios = pd.DataFrame(scenario_results)
print("Resultats de la simulation de drift :\n")
df_scenarios

### Interpretation

- **Aucun drift** : peu ou pas de features detectees — le test confirme que les donnees sont similaires.
- **Drift graduel faible** : quelques features commencent a deriver — signal d'alerte precoce.
- **Drift graduel fort** : une majorite de features driftent — le modele est probablement degrade.
- **Drift soudain** : les 20 premieres features sont fortement decalees — scenario de crise.
- **Feature shift** : ~10% des features changent — probleme cible (ex: un fournisseur de donnees modifie son calcul).

Cela montre que la detection est **proportionnelle a l'intensite du drift**, ce qui est le comportement attendu.


In [ ]:
# Visualiser le rapport Evidently pour le scenario "drift soudain"
drifted_sudden = simulate_drift(prod_sample, drift_type="sudden", intensity=0.3)
report_sudden = Report([DataDriftPreset()])
snap_sudden = report_sudden.run(ref_sample, drifted_sudden)

# Sauvegarder pour reference
snap_sudden.save_html(str(Path("../monitoring/drift_report_sudden.html")))
print("Rapport 'drift soudain' sauvegarde.")

# Afficher le rapport interactif
snap_sudden

---

## 5. Analyse approfondie des features critiques

Toutes les features n'ont pas la meme importance pour le modele. Les **top features** (celles qui influencent le plus les predictions) meritent une surveillance prioritaire.

Voici les 10 features les plus importantes de notre modele LightGBM :


In [ ]:
# Charger le modele et extraire les importances
model = joblib.load(ARTIFACTS_DIR / "model.pkl")
with open(ARTIFACTS_DIR / "feature_names.json") as f:
    feature_names = json.load(f)

importances = sorted(
    zip(feature_names, model.feature_importances_),
    key=lambda x: x[1],
    reverse=True,
)

top_features = [name for name, _ in importances[:10]]
# Garder celles presentes dans les donnees
top_features = [f for f in top_features if f in common_cols]

print("Top features du modele (par importance) :\n")
for i, (name, imp) in enumerate(importances[:10], 1):
    flag = " ← critique" if i <= 3 else ""
    print(f"  {i:2d}. {name:<35s} importance = {imp}{flag}")

In [ ]:
# Analyse de drift feature par feature avec Evidently
top_available = [f for f in top_features if f in ref_sample.columns]

report_top = Report([ValueDrift(column=col) for col in top_available])
snap_top = report_top.run(ref_sample, prod_sample)

# Extraire les resultats
top_results = snap_top.dict()
top_drift_data = []
for m in top_results["metrics"]:
    if m["config"]["type"] == "evidently:metric_v2:ValueDrift":
        top_drift_data.append(
            {
                "Feature": m["config"]["column"],
                "Methode": m["config"]["method"],
                "P-value": f"{m['value']:.6f}",
                "Drift detecte": "OUI"
                if m["value"] < m["config"]["threshold"]
                else "NON",
            }
        )

df_top_drift = pd.DataFrame(top_drift_data)
print("Analyse de drift sur les features les plus importantes du modele :\n")
df_top_drift

In [ ]:
# Visualiser les distributions des top 3 features
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, min(3, len(top_available)), figsize=(16, 4))
if len(top_available) == 1:
    axes = [axes]

for ax, feat in zip(axes, top_available[:3]):
    ax.hist(
        ref_sample[feat].dropna(),
        bins=50,
        alpha=0.6,
        label="Reference",
        color="#1D6A4B",
        density=True,
    )
    ax.hist(
        prod_sample[feat].dropna(),
        bins=50,
        alpha=0.6,
        label="Production",
        color="#8B2D2D",
        density=True,
    )
    ax.set_title(feat, fontsize=12, fontweight="bold")
    ax.legend(fontsize=9)
    ax.set_ylabel("Densite")

plt.suptitle(
    "Distributions comparees — Top Features (Reference vs Production)",
    fontsize=14,
    fontweight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

### Interpretation metier

- **EXT_SOURCE_MEAN / EXT_SOURCE_1 / EXT_SOURCE_3** : ces features sont les **scores externes** fournis par des organismes tiers. Si elles driftent en production, cela peut signifier que le fournisseur a modifie son calcul de score. C'est un signal critique.

- **AMT_CREDIT / AMT_ANNUITY / AMT_GOODS_PRICE** : montants financiers. Un drift ici peut indiquer un changement de segment de clientele (clients plus riches ou plus modestes) ou une evolution economique (inflation).

- **DAYS_BIRTH** : age des clients. Un drift signifierait un changement de la demographie ciblee.

- **PREV_CNT_PAYMENT_MEAN / BUREAU_DAYS_CREDIT_MAX** : historique de credit. Ces features dependent du comportement passe des clients et sont naturellement plus stables.


In [ ]:
# Impact du drift simule sur les features critiques
print("Impact du drift graduel (intensite 0.3) sur les top features :\n")

drifted_gradual = simulate_drift(prod_sample, drift_type="gradual", intensity=0.3)

report_impact = Report([ValueDrift(column=col) for col in top_available])
snap_impact = report_impact.run(ref_sample, drifted_gradual)

impact_results = snap_impact.dict()
for m in impact_results["metrics"]:
    if m["config"]["type"] == "evidently:metric_v2:ValueDrift":
        col = m["config"]["column"]
        pval = m["value"]
        drifted = pval < m["config"]["threshold"]
        status = "DRIFT" if drifted else "OK"
        print(f"  {col:<35s} p-value = {pval:.6f}  [{status}]")

---

## 6. Points de vigilance et recommandations

### Resultats de l'analyse

| Aspect                             | Constat                                                                                                |
| ---------------------------------- | ------------------------------------------------------------------------------------------------------ |
| **Drift baseline (train vs test)** | Faible ou absent — les deux jeux proviennent de la meme source, ce qui est rassurant                   |
| **Robustesse de la detection**     | Evidently detecte correctement les drifts simules, proportionnellement a leur intensite                |
| **Features critiques**             | EXT*SOURCE*\*, AMT_CREDIT, DAYS_BIRTH sont les plus importantes et doivent etre monitorees en priorite |

### Recommandations pour la production

1. **Monitoring regulier** : executer un rapport Evidently chaque semaine (ou chaque lot de N predictions) en comparant les nouvelles donnees a la reference.

2. **Seuils d'alerte** :
   - Si < 10% des features driftent → **surveillance normale**
   - Si 10-30% des features driftent → **alerte moderee**, investigation necessaire
   - Si > 30% des features driftent → **alerte critique**, envisager un re-entrainement

3. **Focus sur les features critiques** : meme si le drift global est faible, un drift sur EXT_SOURCE_MEAN ou AMT_CREDIT (top features) doit declencher une investigation.

4. **Logging des features** : actuellement l'API logue les predictions (probabilite, decision, latence) mais pas les features d'entree. Pour un monitoring complet, il faudrait aussi logger un echantillon des features.

5. **Re-entrainement** : si un drift significatif est confirme sur les features critiques ET que les metriques de performance se degradent, re-entrainer le modele avec des donnees recentes.

### Conformite RGPD

- Les donnees utilisees sont des **features anonymisees** (pas de noms, adresses, etc.)
- Les logs de prediction ne contiennent que l'ID client et le score, pas les features brutes
- Les rapports de drift analysent des **distributions statistiques**, pas des donnees individuelles
- En production, s'assurer que les donnees de reference sont stockees de maniere securisee et avec une duree de retention definie

### Limites

- **Drift ≠ degradation** : un drift statistique ne signifie pas forcement que le modele performe moins bien. Les deux doivent etre surveilles conjointement.
- **Sensibilite au sample size** : avec de grands echantillons, le test KS peut detecter des differences minimes mais statistiquement significatives, sans impact reel sur le modele.
- **Drift simule vs reel** : nos simulations sont artificielles. En production, le drift est souvent plus subtil et multidimensionnel.


In [ ]:
print("Analyse de drift terminee.")
print("\nFichiers generes :")
print("  - monitoring/drift_report_baseline.html  (rapport reference vs production)")
print("  - monitoring/drift_report_sudden.html     (rapport drift soudain simule)")
print(
    "\nOuvrez ces fichiers HTML dans un navigateur pour explorer les rapports interactifs."
)